# Analyzing Trends in Education Within Sub-Saharn Africa

## Importing packages

In [2]:
# import relevant packages here or going forward
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from   sqlalchemy.engine import URL
from   sqlalchemy import text

### Connecting to SQL server

In [17]:
# Write your answer here: 
url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = "postgres") # I installed postgres in a separate class so I have a password

connection = create_engine(url_server)

### Loading the Data

In [31]:
education_data = pd.read_csv('data/WDI_Data.csv')

education_data.to_sql('education_data',
               con = connection,
               if_exists='replace',
               index=False)

95